# Gold Layer: Slot Machine Performance Aggregations

**Notebook:** `01_gold_slot_aggregations`  
**Layer:** Gold (Business)  
**Purpose:** Create business-level aggregations and KPIs for slot performance

---

## Overview

The Gold layer creates business-ready aggregations optimized for analytics and reporting. This notebook creates:

- Daily slot machine performance metrics
- Hourly revenue aggregations
- Machine-level KPIs
- Floor section analytics

In [ ]:
# Configuration
SILVER_TABLE = "silver_slot_telemetry"
GOLD_LAKEHOUSE = "lh_gold"

# Output tables
FACT_DAILY_SLOTS = "fact_daily_slot_performance"
FACT_HOURLY_REVENUE = "fact_hourly_revenue"
DIM_MACHINE = "dim_machine"

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from datetime import datetime

spark = SparkSession.builder.getOrCreate()

In [ ]:
# Read Silver data
df_silver = spark.table(SILVER_TABLE)
print(f"Silver records: {df_silver.count()}")

## Dimension: Machine Master

In [ ]:
# Create machine dimension
df_dim_machine = df_silver.select(
    "machine_id",
    "casino_id",
    "floor_location",
    "denomination"
).distinct() \
    .withColumn("machine_key", sha2(col("machine_id"), 256)) \
    .withColumn("effective_date", current_date()) \
    .withColumn("is_current", lit(True))

df_dim_machine.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(DIM_MACHINE)

print(f"Created {DIM_MACHINE} with {df_dim_machine.count()} machines")

## Fact: Daily Slot Performance

In [ ]:
# Daily aggregation by machine
df_daily = df_silver.filter(col("event_type") == "SPIN") \
    .groupBy(
        "machine_id",
        "casino_id",
        "floor_location",
        "denomination",
        to_date("event_timestamp").alias("play_date")
    ).agg(
        count("*").alias("total_spins"),
        sum("bet_amount").alias("total_coin_in"),
        sum("win_amount").alias("total_coin_out"),
        countDistinct("player_id").alias("unique_players"),
        countDistinct("session_id").alias("total_sessions"),
        max("win_amount").alias("max_single_win"),
        avg("bet_amount").alias("avg_bet_amount"),
        sum(when(col("is_large_win"), 1).otherwise(0)).alias("jackpot_count"),
        sum("jackpot_contribution").alias("total_jackpot_contribution")
    ) \
    .withColumn("net_revenue", col("total_coin_in") - col("total_coin_out")) \
    .withColumn("hold_percentage", 
        when(col("total_coin_in") > 0, 
             (col("total_coin_in") - col("total_coin_out")) / col("total_coin_in") * 100
        ).otherwise(0)) \
    .withColumn("theoretical_hold", lit(8.0))  # 8% theoretical
    
df_daily.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("play_date") \
    .saveAsTable(FACT_DAILY_SLOTS)

print(f"Created {FACT_DAILY_SLOTS}")
df_daily.show(5)

## Fact: Hourly Revenue

In [ ]:
# Hourly revenue by floor section
df_hourly = df_silver.filter(col("event_type") == "SPIN") \
    .withColumn("play_hour", date_trunc("hour", col("event_timestamp"))) \
    .groupBy(
        "casino_id",
        "floor_location",
        "play_hour"
    ).agg(
        count("*").alias("total_transactions"),
        sum("bet_amount").alias("hourly_coin_in"),
        sum("win_amount").alias("hourly_coin_out"),
        countDistinct("machine_id").alias("active_machines"),
        countDistinct("player_id").alias("active_players")
    ) \
    .withColumn("hourly_net_revenue", col("hourly_coin_in") - col("hourly_coin_out")) \
    .withColumn("revenue_per_machine", 
        col("hourly_net_revenue") / col("active_machines"))

df_hourly.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(FACT_HOURLY_REVENUE)

print(f"Created {FACT_HOURLY_REVENUE}")
df_hourly.orderBy(col("hourly_net_revenue").desc()).show(10)

## Summary KPIs

In [ ]:
# Calculate summary KPIs
kpis = df_daily.agg(
    sum("total_coin_in").alias("total_coin_in"),
    sum("total_coin_out").alias("total_coin_out"),
    sum("net_revenue").alias("total_net_revenue"),
    sum("total_spins").alias("total_spins"),
    avg("hold_percentage").alias("avg_hold_pct"),
    sum("jackpot_count").alias("total_jackpots")
).collect()[0]

print("\n" + "="*50)
print("GOLD LAYER KPI SUMMARY")
print("="*50)
print(f"Total Coin In:     ${kpis['total_coin_in']:,.2f}")
print(f"Total Coin Out:    ${kpis['total_coin_out']:,.2f}")
print(f"Net Revenue:       ${kpis['total_net_revenue']:,.2f}")
print(f"Total Spins:       {kpis['total_spins']:,}")
print(f"Avg Hold %:        {kpis['avg_hold_pct']:.2f}%")
print(f"Total Jackpots:    {kpis['total_jackpots']:,}")
print("="*50)